<a href="https://colab.research.google.com/github/jsvan/MaskTests4LanguageModels/blob/main/Summerwinograndeattempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

How does a language model understand words? It represents input text in a rich and complex feature vector. Simple word vectors learn co-occurence probabilities between words, and represent each word in context with every other word. Better embeddings use the maximum space allowed to represent words as far away from each other as possible. 

When Roberta trains on Winogrande, how is it able to differentiate the different words in its task? For example, 
> "The employees threw a [party] and drank so much [alcohol] that they could not go into work the next day. The _ was loud. "

"Party" is represented very differently in vector space than "Alcohol". It should be no problem for the classifier to differentiate those objects. What happens if we mask the objects, thereby removing the representational advantage these words have. Will the classifier get confused between what "mask1" and "mask2" refer to? Do different masking techniques alter accuracy to any noticable degree?

In [ ]:
%%capture
!pip install transformers
!pip install datasets 

In [ ]:
%%capture
from datasets import load_dataset, concatenate_datasets

# You can switch between these two datasets
dataset = load_dataset("winogrande", 'winogrande_debiased')
#dataset = load_dataset("winogrande", 'winogrande_l')

# dataset is dict with keys ['train', 'test', 'validation']
# Each with an enumerable of 
"""
{'answer': '2',
 'option1': 'Kyle',
 'option2': 'Logan',
 'sentence': "Kyle doesn't wear leg warmers to bed, while Logan almost always does. _ is more likely to live in a colder climate."}

"""
# Use Validation instead of Test because Test lacks labels.

In [ ]:
dataset['validation'][4]

{'answer': '1',
 'option1': 'Jeffrey',
 'option2': 'Hunter',
 'sentence': 'At night, Jeffrey always stays up later than Hunter to watch TV because _ wakes up late.'}

In [ ]:
# Textizer
"""
    Each sentence was split on "_" placeholder symbol.
    Each option was concatenated with the second part of the split, thus transforming each example into two text segment pairs.
    Text segment pairs corresponding to correct and incorrect options were marked with True and False labels accordingly.
    Text segment pairs were shuffled thereafter.

"""

from datasets import Dataset

def prepare_data(dataset):

  # internal function
  def prep_ds(dataset):
    sentences, answers, o1, o2 = [], [], [], []
    for p in dataset:
      s1 = p['option1'].join(p['sentence'].split('_'))
      s2 = p['option2'].join(p['sentence'].split('_'))
      a1 = int(p['answer'] == '1')
      a2 = int(p['answer'] == '2')

      
      sentences.append(s1)
      answers.append(a1)
      sentences.append(s2)
      answers.append(a2)
      o1.append(p['option1'])
      o2.append(p['option2'])
      o1.append(p['option1'])
      o2.append(p['option2'])

    return {'sentence':sentences, 'labels':answers, 'option1':o1, 'option2':o2}
    # end internal function

  train = prep_ds(dataset["train"])
  test = prep_ds(dataset["validation"])
  trainds = Dataset.from_dict( train ).shuffle()
  testds = Dataset.from_dict( test ).shuffle()

  return {"train":trainds, "test":testds, "name":"Standard Dataset"}





def mask_copy(dataset):
  sentences = []
  toprint = 10

  for p in dataset:
    if toprint > 0:
      print(f"[{p['option1']}], [{p['option2']}], [{p['sentence']}]")

    sentences.append(p['sentence'].replace(p['option1'], 'option1').replace(p['option2'], 'option2'))
    
    if toprint > 0:
      print(sentences[-1])
      toprint -= 1

  build = {'sentence':sentences, 'labels':dataset['labels'], 'option1':dataset['option1'], 'option2':dataset['option2']}
  return Dataset.from_dict(build) # DON'T SHUFFLE





def mask_datasets(dataset):
  return {"train":mask_copy(dataset['train']), "test":mask_copy(dataset['test']), "name":"Masked Dataset"}




In [ ]:
%%capture
#dicts of {'train', 'test'}
std_datasets = prepare_data(dataset)

masked_datasets = mask_datasets(std_datasets)

If we peak at the two datasets, we can see that the masking did indeed work.

In [ ]:
std_datasets['train']['sentence'][:5]

['Robert lent the book to Ian for the reason that Robert already read the book.',
 'The woman needed to decide between the jacket and the sweater and ultimately chose the sweater because she disliked the color of the yarn.',
 'Dennis won a swimming race to Donald because Donald had shorter and weaker arms and legs.',
 'I had to communicate the news over the phone or at the restaurant tonight. I told them at the phone even though I wanted them to enjoy their evening.',
 'Jessica often experiences severe nausea, Victoria does not therefore Victoria often rides big roller coasters.']

In [ ]:
masked_datasets['test']['sentence'][:5]

["option1 went to option2's house to play with the new dog, but there was no answer. option1 was at the park.",
 'No one would have noticed the option1 on that option2 because the option2 is small.',
 'Eating spicy foods better suited option1 and not option2 because option1 never got acid reflux from salsa.',
 'Timmy bought a option1 for his cat so he could take him on the option2 but the option1 was too small.',
 'option1 has dark lips unlike option2 due to option2 forgetting to put on chapsticks at night.']

This upcoming section looks at how the pretrained model performs on winogrande standard vs. the masking technique 'option1' and 'option2'. 

This model is a Roberta-Large model that was trained on winogrande_xl. On the winogrande_m dataset:
the standard dataset receives 85% accuracy, and
the masked dataset receives 80% accuracy. 

When we use the winogrande_debiased dataset, results fall to %69 and 68% respectively. Because these are so bad, let's retune the model on the debiased_train set. 

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from pprint import pprint
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/roberta-large-winogrande", model_max_length=64)

def test_datasets(tokenizer, std_datasets, masked_datasets, model=False,):

  print("Testing 1 2 3 ...")
  delete = False
  if not model:
    model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/roberta-large-winogrande")
    delete = True

  elif isinstance(model, str):
    delete = True
    with torch.no_grad():
      torch.cuda.empty_cache()

    model = torch.load(model) # open(model, "rb"))

    with torch.no_grad():
      torch.cuda.empty_cache()
  try:
    for ds in (std_datasets, masked_datasets):
      print("Performing", ds["name"])
      
      #combinedtraintest = concatenate_datasets([ds['train'], ds['test']])
      #encoded_train = combinedtraintest.map(lambda examples: tokenizer(examples['sentence'], padding='max_length'), batched=True) # , return_tensors='pt'
      encoded_test = ds['test'].map(lambda examples: tokenizer(examples['sentence'], padding='max_length'), batched=True)

      #encoded_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
      encoded_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
      #dataloader_train = torch.utils.data.DataLoader(encoded_train, batch_size=32)
      dataloader_test = torch.utils.data.DataLoader(encoded_test, batch_size=32)

      device = 'cuda' if torch.cuda.is_available() else 'cpu' 
      #model.train().to(device)
      model.to(device)
      #optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)

      correct = 0
      total = 0
      for i, batch in enumerate(tqdm(dataloader_test)):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        #for oo, lab in zip(outputs.logits, batch['labels'] ):
        #  print(oo.argmax().item(), lab.items())
        correct += sum((int(x.argmax().item() == y.item()) for x, y in zip(outputs.logits, batch['labels'])))
        total += len(batch['labels'])
        
        
      print("Score", correct / total, '/ 1.00')

    if delete:
      del model
      print("Deleted model")

  except Exception as e:
    del model
    print(e, e.__str__)
    print("Deleted model")


# test_datasets(tokenizer, std_datasets, masked_datasets)

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

We'll train a model on the unmasked debiased data train set for one epoch. 

On the first epoch, the testdataset achieves 75% accuracy and the masked dataset 70% accuracy. 

Further epochs see no change (tested with 3)

In [ ]:

def train_model(trainingset, compareset, tokenizer, model=False, copy=False):
  if not model:
    model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/roberta-large-winogrande")
  elif copy: # ie, if copy and model
    print('copy')
    with torch.no_grad():
      torch.cuda.empty_cache()
    model = torch.load(model) # open(model, "rb"))
    with torch.no_grad():
      torch.cuda.empty_cache()
    #model_copy = type(model)() # get a new instance
    #model_copy.load_state_dict(model.state_dict()) # copy weights and stuff
    #model = model_copy
    print('finished?')
  try:
    encoded_train = trainingset['train'].map(lambda examples: tokenizer(examples['sentence'], padding='max_length'), batched=True) # , return_tensors='pt'
    encoded_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    dataloader_train = torch.utils.data.DataLoader(encoded_train, batch_size=32)

    device = 'cuda' if torch.cuda.is_available() else 'cpu' 
    model.train().to(device)
    optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)

    for epoch in range(1):
      print("Epoch", epoch)
      correct = 0
      total = 0
      for i, batch in enumerate(tqdm(dataloader_train)):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
          print(f" loss: {loss}")
        #for oo, lab in zip(outputs.logits, batch['labels'] ):
        #  print(oo.argmax().item(), lab.items())
        correct += sum((int(x.argmax().item() == y.item()) for x, y in zip(outputs.logits, batch['labels'])))
        total += len(batch['labels'])
        
      print("Score", correct / total, '/ 1.00')
      test_datasets(tokenizer, trainingset, compareset, model)
      model.train().to(device)
  except Exception as e:
    del model
    print(e, e.__str__)
    print("deleted model")
  return model




In [ ]:

# This model will become the base model for everything we build upon, 
# so we will save it to disk to load it fresh for each experiment. 
debiased_tuned_model = train_model(std_datasets, masked_datasets, tokenizer)
torch.save(debiased_tuned_model, f="debiased_model")
debiased_tuned_model = None

Downloading:   0%|          | 0.00/820 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Parameter 'function'=<function train_model.<locals>.<lambda> at 0x7f57134b7440> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/19 [00:00<?, ?ba/s]

Epoch 0


...

100%|██████████| 80/80 [00:18<00:00,  4.30it/s]

Score 0.7505919494869772 / 1.00
Performing Masked Dataset


  0%|          | 0/3 [00:00<?, ?ba/s]

100%|██████████| 80/80 [00:18<00:00,  4.31it/s]


Score 0.7087608524072613 / 1.00


Does training it on the masked testset change anything?

In [ ]:
#Clear the GPU RAM...

with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
#masked_tuned_model = train_model(masked_datasets, std_datasets, tokenizer)


A pre-tuned model on Winogrande_xl that's tuned on the masked-debiased training set gets slightly better accuracy on the masked test set. (73.2% accuracy for standard, 73.8% for masked). 

The same holds for if we use the winogrande_l dataset instead of debiased. On Winogrande_l standard we get 72% accuracy and 71.5% on the masked. 

What happens if we tune the masked set on top of the tuned standard debiased model?

In [ ]:
#with torch.no_grad():
#    torch.cuda.empty_cache()

In [ ]:
#debiased_masked_tuned_model = train_model(masked_datasets, std_datasets, tokenizer, model = debiased_tuned_model)


In this case, the standard debiased dataset achieves 73.6% accuracy and the masked dataset 73.4% accuracy. Practically the same. 

Let's try a different masking style. What happens if we cover up one of the Options with an [unknown] mask? Can the model deal with and identify objecthood of a masked object?

Take the following example sentence:

{
  "sentence": "The [plant] took up too much room in the [urn], because the [plant] was small.",
  "label": false
}

There are two ways to test the model. If we mask the plant, then there are two references to the same masked object, and the True/False question directly references that object. If we mask the urn, then the contextual object is hidden, but is never the subject of inquery. Let's test covering up each at a time. The first test is more interesting, but might as well see if we find anything.

In [ ]:
import re

def mask_copy_1(dataset, unk):
  sentences = []
  toprint = 5

  for p in dataset:
    # This is annoying because option1 can be substrings of other words, usually option2. They can also have uppercase letters. 
    # If one is a substring, then I will cover up the larger word with a temporary mask to not confuse anything else.
    sentence = p['sentence']
    option1, option2 = re.compile(p['option1'], re.IGNORECASE), re.compile(p['option2'], re.IGNORECASE)
    maskedoption1, maskedoption2 = "OPTION_ONE", "OPTION_TWO"
    first_search, second_search, first_mask, second_mask = None, None, None, None

    # "table" in "tablecloth" --> cover bigger one
    if len(p['option1']) > len(p['option2']):
      # cover option1 first
      first_search, second_search = option1, option2
      first_mask, second_mask = maskedoption1, maskedoption2
    else:
      first_search, second_search = option2, option1
      first_mask, second_mask = maskedoption2, maskedoption1
    
    sentence = first_search.sub(first_mask, sentence) #Mask the longer word with OPTION_ mask
    sentence = second_search.sub(second_mask, sentence) #then the shorter one

    # IT gets kinda confusing which word now to <IGNORE> because it's language and some words appear more than two, more than three, times. 
    # I think the smartest approach is to assume the final word used is the question word and mask that one. 

    # Find final word used
    # maskedoption1 is the final word
    if sentence.rfind(maskedoption1) > sentence.rfind(maskedoption2):
      # IGNORE final word
      # Convert other word back to original
      sentence = sentence.replace(maskedoption1, unk)
      sentence = sentence.replace(maskedoption2, p['option2'])
    else:
      sentence = sentence.replace(maskedoption2, unk)
      sentence = sentence.replace(maskedoption1, p['option2'])


    if toprint > 0:
      print(f"[{p['option1']}], [{p['option2']}], [{p['sentence']}]")

    sentences.append(sentence)
    
    if toprint > 0:
      print(sentences[-1])
      toprint -= 1

  build = {'sentence':sentences, 'labels':dataset['labels'], 'option1':dataset['option1'], 'option2':dataset['option2']}
  return Dataset.from_dict(build) # DON'T SHUFFLE





def mask_datasets_1(dataset, tokenizer):
  unk = tokenizer.unk_token
  return {"train":mask_copy_1(dataset['train'], unk), "test":mask_copy_1(dataset['test'], unk), "name":"Double <Unk> Masked Dataset"}

In [ ]:
#unk_datasets = mask_datasets_1(std_datasets, tokenizer)



In [ ]:
#test_datasets(tokenizer, std_datasets, unk_datasets, debiased_tuned_model)

The [unk] tokens reduce performance from 75% accuracy to 65% accuracy. Let's tune the model on the [unk] masked dataset and see if it redeems itself. 

In [ ]:
#unk_masked_tuned_model = train_model(unk_datasets, std_datasets, tokenizer)


On training on the debiased [unk] masked set, we perform with 73% accuracy. On the standard set we perform 69% accuracy. Strange the standard dataset decreased. That may mean there is some bias in the dataset I created where it learns to do tricks. 

Let's try masking the single word. 

In [ ]:
import re


# This masked <unk> tokens on the option which is NOT involved in the question. 
# If the option involved in the question is wrong, it will need to identify the <unk> token as having importance.
def mask_copy_2(dataset, unk):
  sentences = []
  toprint = 5

  for p in dataset:
    # This is annoying because option1 can be substrings of other words, usually option2. They can also have uppercase letters. 
    # If one is a substring, then I will cover up the larger word with a temporary mask to not confuse anything else.
    sentence = p['sentence']
    option1, option2 = re.compile(p['option1'], re.IGNORECASE), re.compile(p['option2'], re.IGNORECASE)
    maskedoption1, maskedoption2 = "OPTION_ONE", "OPTION_TWO"
    first_search, second_search, first_mask, second_mask = None, None, None, None

    # "table" in "tablecloth" --> cover bigger one
    if len(p['option1']) > len(p['option2']):
      # cover option1 first
      first_search, second_search = option1, option2
      first_mask, second_mask = maskedoption1, maskedoption2
    else:
      first_search, second_search = option2, option1
      first_mask, second_mask = maskedoption2, maskedoption1
    
    sentence = first_search.sub(first_mask, sentence) #Mask the longer word with OPTION_ mask
    sentence = second_search.sub(second_mask, sentence) #then the shorter one

    # IT gets kinda confusing which word now to <IGNORE> because it's language and some words appear more than two, more than three, times. 
    # I think the smartest approach is to assume the final word used is the question word and mask that one. 

    # Find final word used
    # maskedoption1 is NOT the final word
    if sentence.rfind(maskedoption1) < sentence.rfind(maskedoption2):
      # IGNORE final word
      # Convert other word back to original
      sentence = sentence.replace(maskedoption1, unk)
      sentence = sentence.replace(maskedoption2, p['option2'])
    else:
      sentence = sentence.replace(maskedoption2, unk)
      sentence = sentence.replace(maskedoption1, p['option2'])


    if toprint > 0:
      print(f"[{p['option1']}], [{p['option2']}], [{p['sentence']}]")

    sentences.append(sentence)
    
    if toprint > 0:
      print(sentences[-1])
      toprint -= 1

  build = {'sentence':sentences, 'labels':dataset['labels'], 'option1':dataset['option1'], 'option2':dataset['option2']}
  return Dataset.from_dict(build) # DON'T SHUFFLE





def mask_datasets_2(dataset, tokenizer):
  unk = tokenizer.unk_token
  return {"train":mask_copy_2(dataset['train'], unk), "test":mask_copy_2(dataset['test'], unk), "name":"Single <Unk> Masked Dataset"}

In [ ]:
unk_dataset_2 = mask_datasets_2(std_datasets, tokenizer)

#test_datasets(tokenizer, std_datasets, unk_dataset_2, debiased_tuned_model)

[Robert], [Ian], [Robert lent the book to Ian for the reason that Robert already read the book.]
Ian lent the book to <unk> for the reason that Ian already read the book.
[sweater], [jacket], [The woman needed to decide between the jacket and the sweater and ultimately chose the sweater because she disliked the color of the yarn.]
The woman needed to decide between the <unk> and the jacket and ultimately chose the jacket because she disliked the color of the yarn.
[Dennis], [Donald], [Dennis won a swimming race to Donald because Donald had shorter and weaker arms and legs.]
<unk> won a swimming race to Donald because Donald had shorter and weaker arms and legs.
[phone], [restaurant], [I had to communicate the news over the phone or at the restaurant tonight. I told them at the phone even though I wanted them to enjoy their evening.]
I had to communicate the news over the restaurant or at the <unk> tonight. I told them at the restaurant even though I wanted them to enjoy their evening.


The standard set achieves 75% accuracy and the single \<unk\> 68%. 

Let's train the model on the single \<unk\>. It achieves 75% on standard and 73% on \<unk\>, which is 5 percentage points higher


In [ ]:
with torch.no_grad():
  torch.cuda.empty_cache()

In [ ]:

#train_model(unk_dataset_2, std_datasets, tokenizer, model="debiased_model", copy = True)

What happens if we try a bunch of stupid masks? Can we make masks so linguistically non-sensical that the model performance drops?

We can try such masks as 



In [ ]:
stupid_tries = [
  ("dog", "doggy"),
  ("red", "blue"),
  ("flavor", "flavour"),
  ('A', 'B'),
  ('X', 'Y'),
  ('1', '2'),
  ('first', 'second'),
  ('alpha', 'beta'),
  ('#', '@'),
  ('primero', 'secundo'),
  ('Alice', 'Bob'),
  ('_', '__')
  
]


In [ ]:
import re


# This masked <unk> tokens on the option which is NOT involved in the question. 
# If the option involved in the question is wrong, it will need to identify the <unk> token as having importance.
def stupid_masking(dataset, mask1, mask2):
  sentences = []
  toprint = 5

  for p in dataset:
    # This is annoying because option1 can be substrings of other words, usually option2. They can also have uppercase letters. 
    # If one is a substring, then I will cover up the larger word with a temporary mask to not confuse anything else.
    sentence = p['sentence']
    option1, option2 = re.compile(p['option1'], re.IGNORECASE), re.compile(p['option2'], re.IGNORECASE)
    maskedoption1, maskedoption2 = "OPTION_ONE", "OPTION_TWO"
    first_search, second_search, first_mask, second_mask = None, None, None, None

    # "table" in "tablecloth" --> cover bigger one
    if len(p['option1']) > len(p['option2']):
      # cover option1 first
      first_search, second_search = option1, option2
      first_mask, second_mask = maskedoption1, maskedoption2
    else:
      first_search, second_search = option2, option1
      first_mask, second_mask = maskedoption2, maskedoption1
    
    sentence = first_search.sub(first_mask, sentence) #Mask the longer word with OPTION_ mask
    sentence = second_search.sub(second_mask, sentence) #then the shorter one

    # IT gets kinda confusing which word now to <IGNORE> because it's language and some words appear more than two, more than three, times. 
    # I think the smartest approach is to assume the final word used is the question word and mask that one. 

    # IGNORE final word
    # Convert other word back to original
    sentence = sentence.replace(maskedoption1, mask1)
    sentence = sentence.replace(maskedoption2, mask2)


    if toprint > 0:
      print(f"[{p['option1']}], [{p['option2']}], [{p['sentence']}]")

    sentences.append(sentence)
    
    if toprint > 0:
      print(sentences[-1])
      toprint -= 1

  build = {'sentence':sentences, 'labels':dataset['labels'], 'option1':dataset['option1'], 'option2':dataset['option2']}
  return Dataset.from_dict(build) # DON'T SHUFFLE





def stupid_datasets(dataset, tokenizer, masks):
  mask1, mask2 = masks
  return {"train":stupid_masking(dataset['train'], mask1, mask2), "test":stupid_masking(dataset['test'], mask1, mask2), "name":f"({mask1}, {mask2}) Masked Dataset"}

In [ ]:


for masks in stupid_tries[3:]:
  stupid_ds = stupid_datasets(std_datasets, tokenizer, masks)
  test_datasets(tokenizer, std_datasets, stupid_ds, model="debiased_model")
  train_model(stupid_ds, std_datasets, tokenizer, model="debiased_model", copy = True)


[Robert], [Ian], [Robert lent the book to Ian for the reason that Robert already read the book.]
A lent the book to B for the reason that A already read the book.
[sweater], [jacket], [The woman needed to decide between the jacket and the sweater and ultimately chose the sweater because she disliked the color of the yarn.]
The woman needed to decide between the B and the A and ultimately chose the A because she disliked the color of the yarn.
[Dennis], [Donald], [Dennis won a swimming race to Donald because Donald had shorter and weaker arms and legs.]
A won a swimming race to B because B had shorter and weaker arms and legs.
[phone], [restaurant], [I had to communicate the news over the phone or at the restaurant tonight. I told them at the phone even though I wanted them to enjoy their evening.]
I had to communicate the news over the A or at the B tonight. I told them at the A even though I wanted them to enjoy their evening.
[Jessica], [Victoria], [Jessica often experiences severe n

  0%|          | 0/3 [00:00<?, ?ba/s]

100%|██████████| 80/80 [00:18<00:00,  4.32it/s]

Score 0.7375690607734806 / 1.00
Performing (A, B) Masked Dataset


  0%|          | 0/3 [00:00<?, ?ba/s]

...

 43%|████▎     | 251/578 [02:58<03:52,  1.41it/s]

 loss: 0.04349224641919136


 45%|████▌     | 261/578 [03:05<03:45,  1.40it/s]

 loss: 0.16291408240795135


 45%|████▌     | 262/578 [03:06<03:44,  1.41it/s]

Results:

| Dataset | Accuracy (before training) | Accuracy (after training on silly set) |
|:-------:|:------------------:|:----------------:|
|Standard set| 75% | 75% |
|(dog, doggy)| 65% | 73% | 
|(red, blue) | 71% | 73% |
|(darling, sweetheart)|  |  |
| (A, B) | 72% | 73% |
| (X, Y) | 72% | 74% |
| (1, 2) | 71% | 74% |
|(first, second)| 69% | 73% |
|(alpha, beta)| 72% |  |
|(#, @)|  |  |
|(primero, secundo)|  |  |
|(Alice, Bob)|  |  |
|(flavor, flavour)|  |  |


